In [ ]:
from keras.applications.convnext import LayerScale
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix, f1_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

from classification_metrics import plot_confusion_matrix, print_confidence_intervals, get_performance_metrics, bootstrap_auc, plot_calibration_curve, roc_auc_score, get_accuracy, get_prevalence, get_sensitivity, get_specificity, get_ppv, get_npv 
from data_preparation import prepare_data

In [ ]:
TARGET_HEIGHT = 640
TARGET_WIDTH = 640

## Load data

In [ ]:
X_val, y_val, valid_labels_df, targets = prepare_data()

In [ ]:
#get the first 5 images
paths =  valid_labels_df.path[:5]
labels = valid_labels_df.feature_string[:5]

fig, m_axs = plt.subplots(1, len(labels), figsize = (20, 10))
#show the images and label them
for ii, c_ax in enumerate(m_axs):
    c_ax.imshow(np.asarray(Image.open(paths[ii])), cmap='gray')
    c_ax.set_title(labels[ii])
plt.show()

## Load model

In [ ]:
MODEL_NAME = 'pretrain_model_ConvNeXtBase_w_ClssWgt_01-0.3616.h5'

In [ ]:
model_path = f'../models/{MODEL_NAME}'

In [ ]:
model = tf.keras.models.load_model(model_path, custom_objects={'LayerScale': LayerScale})

In [ ]:
model.summary()

## Store predictions

Store all predictions in a dataframe

CAUTION! Make sure the preprocessing matches what was used during model training 

In [ ]:
def convert_image_to_array(path):
    img = np.asarray(Image.open(path), dtype=np.float32)
    img = np.stack((img,)*3, axis=-1)
    img /= 255.
    img = tf.image.resize_with_pad(img, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH)
    return img

In [ ]:
def model_predict(path, model):
    x = convert_image_to_array(path=path)
    x = np.expand_dims(x, axis=0)
    return model.predict(x)

In [ ]:
pred_columns = [col_name + '_pred' for col_name in targets]
target_columns = [col_name + '_label' for col_name in targets]

In [ ]:
all_model_preds = pd.DataFrame(0, index=np.arange(len(X_val)), columns=pred_columns)

In [ ]:
for i, path in enumerate(X_val):
    if i % 10 == 0:
        print(f'{i} out of {len(X_val)}')
    all_model_preds.iloc[i, :] = model_predict(path=path, model=model)[0]

In [ ]:
all_model_preds

In [ ]:
results = pd.DataFrame(data=y_val, columns=target_columns)
results = pd.concat([results, all_model_preds], axis=1)

In [ ]:
y = results[target_columns].values
pred = results[pred_columns].values

In [ ]:
plt.xticks(rotation=90)
plt.bar(x = target_columns, height= y.sum(axis=0))
plt.show()

## Calculate metrics

In [ ]:
get_performance_metrics(y, pred, target_columns, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score, f1=f1_score)

In [ ]:
statistics = bootstrap_auc(y, pred, target_columns)

In [ ]:
print_confidence_intervals(target_columns, statistics)

In [ ]:
plot_calibration_curve(y, pred)

In [ ]:
model_performance_df = get_performance_metrics(y, pred, target_columns, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

## Save results to disk

In [ ]:
model_performance_df

In [ ]:
model_performance_df.to_csv(f'model_performance/test_metrics_{MODEL_NAME}.csv')